In [ ]:
prefix_path = "/kaggle/input/jane-street-market-prediction/"
train_path = prefix_path + "train.csv"
examples = pd.read_csv(train_path)

In [ ]:
# 筛选待使用的特征
cnt = len(examples)
feature_percent = examples.isnull().sum()/cnt
feature_percent = feature_percent.apply(lambda x:x<0.1)
train_feature = []
for col in feature_percent.keys():
    if col not in ["date","weight","ts_id","resp_1", "resp_2", "resp_3","resp_4", "resp","label"] and feature_percent[col]:
        train_feature.append(col)

# 构建训练样本
from sklearn.model_selection import train_test_split

# 构建标签
def CreateLabel(x):
    count = 0
    value = 0
    for feature in ["resp","resp_1","resp_2","resp_3","resp_4"]:
        if not math.isnan(x[feature]):
            count += 1
            value += x[feature]
    if count == 0:
        return np.nan
    else:
        return value/count
    
examples["label"] = examples.apply(CreateLabel,axis=1)
      
# 抽取特征，并对缺失值处理
examples.dropna(axis=0, how='any',subset=['label'],inplace=True)

# 使用均值填充特征
def GetMean(features,df):
    mean_map = dict()
    for feature in features:
        mean_map[feature] = df[feature].mean()
    return mean_map

label = examples["label"]
mean_map = GetMean(train_feature,examples)
for feature in train_feature:
    examples[feature].fillna(mean_map[feature],inplace=True)
train = examples[train_feature]
# 拆分数据
train_set,eval_set,train_label,eval_label = train_test_split(train,label,test_size=0.2)
print(len(train_set))

In [ ]:
# 构建训练模型
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(train_set, train_label)


# 预测
train_pred = linreg.predict(train_set)
eval_pred = linreg.predict(eval_set)

# 衡量模型效果
from sklearn import metrics
import numpy as np
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(train_label, train_pred))
print("MSE:",metrics.mean_squared_error(eval_label, eval_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_label, train_pred)))
print("RMSE:",np.sqrt(metrics.mean_squared_error(eval_label, eval_pred)))

In [1]:
# # 提交预测
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    # 获取特征，并进行处理
    select_df = test_df[train_feature]
    for feature in train_feature:
        select_df[feature].fillna(mean_map[feature],inplace=True)
    reward = linreg.predict(select_df)
    if reward > 0:
        sample_prediction_df.action = 1
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)

ModuleNotFoundError: No module named 'janestreet'